#### KPI 4: CO2 emissions (top down)
---
Combined data frame for every entity in use case with:
- grey_energy [kWh] -> to be multiplied by co2 emission factor
- grey_energy [%] -> share grey energy consumed (1 - share renewable)

In [2]:
import os
import pandas as pd

from functions_topdown_v2 import *

home_path_new = r'C:\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\07_Arbeitsordner\04_Ergebnisse\20220920_Ergebnisse_4_Wochen\new_results_2023'
results_new = r'C:\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\07_Arbeitsordner\04_Ergebnisse\20220920_Ergebnisse_4_Wochen\new_results_2023\results'

home_path_old = r'C:\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\07_Arbeitsordner\04_Ergebnisse\20220920_Ergebnisse_4_Wochen'
results_old = r'C:\Users\joche\FIM Kernkompetenzzentrum\Projekt VIdES - Dokumente\General\07_Arbeitsordner\04_Ergebnisse\20220920_Ergebnisse_4_Wochen\results'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### Instructions how to use function ```co2_calcuation```:
- enter the relevant simulation data directory for old results (```home_path_old```) or new results (```home_path_new```)
- enter the relevant use case number as an integer
- save the result in a pandas dataframe
- save the dataframe as a xlsx-file to old results folder (path: ```results_old```) or new results folder (path: ```results_new```)

In [65]:
# calculate co2 emissions for old data, use case 4
df4 = co2_calculation(home_path_old, use_case_nr=4)
df4

,region-1-ec0_grey_energy [kWh],region-1-ec0_grey_energy [%],region-1-ec1_grey_energy [kWh],region-1-ec1_grey_energy [%],region-1-ec2_grey_energy [kWh],region-1-ec2_grey_energy [%],region-1-ec3_grey_energy [kWh],region-1-ec3_grey_energy [%],region-1-ec4_grey_energy [kWh],region-1-ec4_grey_energy [%],...,region-3_grey_energy [kWh],region-3_grey_energy [%],region-4_grey_energy [kWh],region-4_grey_energy [%],region-5_grey_energy [kWh],region-5_grey_energy [%],region-6_grey_energy [kWh],region-6_grey_energy [%],germany_grey_energy [kWh],germany_grey_energy [%]
2030-01-01T00:00,-126.140086,0.0,29.095249,0.622659,29.036146,0.622659,31.667665,0.622659,32.302224,0.622659,...,54.315284,0.397341,81.420262,0.412125,121.670530,0.473123,223.060523,0.593280,770.863572,0.486775
2030-01-01T01:00,-121.633744,0.0,29.301829,0.638371,31.344534,0.638371,34.807542,0.638371,33.389546,0.638371,...,56.953928,0.410524,92.040916,0.437648,128.731636,0.492176,226.360310,0.597113,808.200861,0.497900
2030-01-01T02:00,-119.323853,0.0,29.593071,0.631698,30.187390,0.631698,30.913359,0.631698,33.374001,0.631698,...,52.434191,0.402946,90.228321,0.434499,122.173945,0.485521,208.066239,0.583620,767.327944,0.488968
2030-01-01T03:00,-119.464280,0.0,29.396984,0.648326,29.323325,0.648326,30.634198,0.648326,34.788689,0.648326,...,54.225162,0.416763,92.627993,0.449118,131.028267,0.507536,209.629840,0.595109,806.917892,0.506469
2030-01-01T04:00,-122.984529,0.0,30.260041,0.678785,34.318704,0.678785,37.419718,0.678785,36.922576,0.678785,...,65.640515,0.450741,105.736089,0.479289,147.227683,0.543971,236.115269,0.619793,920.101311,0.538824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2030-10-07T19:00,-230.799438,0.0,34.849666,0.141770,32.643932,0.141770,33.354044,0.141770,36.756524,0.141770,...,-61.245996,0.057233,-64.909574,0.059719,56.445719,0.083622,173.833062,0.113193,182.584008,0.086140
2030-10-07T20:00,-224.197998,0.0,31.395628,0.123741,29.315987,0.123741,31.078270,0.123741,35.998404,0.123741,...,-68.157047,0.048206,-62.781967,0.051793,53.157693,0.072989,169.151213,0.101282,148.976133,0.074933
2030-10-07T21:00,-214.679024,0.0,32.542955,0.176952,29.439668,0.176952,30.463604,0.176952,35.556664,0.176952,...,-61.660294,0.070234,-39.000148,0.079070,59.463974,0.106257,174.015493,0.146394,214.617216,0.112208
2030-10-07T22:00,-213.491169,0.0,33.153385,0.155432,27.976335,0.155432,29.648870,0.155432,33.311183,0.155432,...,-56.485650,0.063406,-49.056341,0.067852,53.147752,0.091219,160.119820,0.122263,189.742061,0.095025


In [ ]:
# save dataframe to xlsx-file in old results path
df4.to_excel(os.path.join(results_old, 'use_case_4_topdown.xlsx'))